### IMPORT

In [ ]:
# Importing the required libraries
import torch # Import the Pytorch library
import torchvision # Import the torchvision library
from torchvision import datasets, transforms # Import the transforms module from torchvision


import numpy as np
from PIL import Image # Import the Image module from the Python Imaging Library (PIL)
import matplotlib.pyplot as plt

import urllib # Import the urllib library for URL handling
import sys
from tqdm import tqdm
from customDataset import ISICDataset

# HELPER FUNCTIONS
from data_exploration_helper import dataset_overview

In [4]:
# Training set 2018
TRAIN_2018_LABELS: str = "./data/ISIC2018_Training_GroundTruth.csv"
TRAIN_2018_ROOT_DIR: str = "./data/ISIC2018_Training_Input"

TEST_2018_LABELS: str = "./data/ISIC2018_Validation_GroundTruth.csv"
TEST_2018_ROOT_DIR: str = "./data/ISIC2018_Validation_Input"

# Dataset 2019 - has not been split into train and test
DATASET_2019_LABELS: str = "./data/ISIC_2019_Training_GroundTruth.csv"
DATASET_2019_ROOT_DIR: str = "./data/ISIC_2019_Training_Input"

# Define image pre-processing steps
preprocess_inceptionv3 = transforms.Compose([
    transforms.ToPILImage(), # Removes error
    transforms.Resize(299), # Resize the image to 299x299 pixels
    transforms.CenterCrop(299), # Crop the image to 299x299 pixels (removing any extra pixels)
    transforms.ToTensor(), # Convert the image to a Pytorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize the image using the pre-trained model's mean and standard deviation
])

# Training set 2018 - custom class
train_dataset_2018 = ISICDataset(
    csv_file=TRAIN_2018_LABELS, 
    root_dir=TRAIN_2018_ROOT_DIR, 
    transform=preprocess_inceptionv3,
    image_file_type="jpg",
    nrows=5000 # defines the number of rows used, utilized this for testing purposes
    )


In [5]:


# Define the data loader
data_loader = torch.utils.data.DataLoader(train_dataset_2018, batch_size=32, shuffle=True)


# Load the pretrained Inception v3 model
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)

# Freeze the model parameters to prevent backpropagation
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer with a new layer that matches the number of classes in the dataset
num_classes = len(train_dataset_2018.annotations.columns)-1
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Train the model
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

for epoch in tqdm(range(5)):
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        labels = torch.tensor(labels, dtype=torch.float)
        optimizer.zero_grad()
        outputs, x = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print('Epoch {} loss: {:.4f}'.format(epoch + 1, running_loss / (i + 1)))

print('Finished training')



Using cache found in /Users/fritt/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/fritt/anaconda3/envs/adv_mit/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/fritt/anaconda3/envs/adv_mit/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/5 [00:00<?, ?it/s]/var/folders/pc/5ytjzw0165n86jc4_pv2802c0000gn/T/ipykernel_89272/2351522177.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().

Epoch 1 loss: 1.0401


 40%|████      | 2/5 [18:06<26:47, 535.89s/it]

Epoch 2 loss: 0.8941


 60%|██████    | 3/5 [27:01<17:51, 535.67s/it]

Epoch 3 loss: 0.8390


 80%|████████  | 4/5 [37:19<09:27, 567.89s/it]

Epoch 4 loss: 0.8203


100%|██████████| 5/5 [48:07<00:00, 577.49s/it]

Epoch 5 loss: 0.7912
Finished training


In [6]:
# Test set 2018 - custom class
test_dataset_2018 = ISICDataset(
    csv_file=TEST_2018_LABELS, 
    root_dir=TEST_2018_ROOT_DIR, 
    transform=preprocess_inceptionv3,
    image_file_type="jpg",
    # nrows=200 # defines the number of rows used, utilized this for testing purposes
    )

In [7]:
# Load the test set
test_dataset = test_dataset_2018 # Define the test set in the same way as the training set
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data in test_loader:
        inputs, labels = data
        labels = torch.tensor(labels, dtype=torch.float)
        outputs = model(inputs)
        labels = torch.argmax(labels)
        predicted = torch.argmax(outputs.data)
        total += 1
        if labels==predicted:
            correct += 1
    accuracy = 100 * correct / total
    print('Accuracy of the model on the test set: {:.2f}%'.format(accuracy))


/var/folders/pc/5ytjzw0165n86jc4_pv2802c0000gn/T/ipykernel_89272/1444349468.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.float)


Accuracy of the model on the test set: 69.43%
